In [3]:
import pandas as pd
import basicdescriptives_mod as bd
import re
import string
from unidecode import unidecode
from fuzzywuzzy import fuzz
import numpy as np
import locale

In [4]:
df = pd.read_csv('../data/stage/db_stage_infousers.csv')

In [5]:
df.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'DÍAS MORA', 'Cuotas en mora',
       'ESTRATO', 'INGRESOS MENSUALES', 'GASTOS MENSUALES',
       'INGRESOS ADICIONALES', 'PERSONAS A CARGO', 'NUMERO DE HIJOS',
       'NUM.CREDITOS SOLICITADOS', 'FECHA DESEMBOLSO', 'PRÓXIMA FECHA PAGO',
       'TIPO EMPLEO', 'CIUDAD RESIDENCIA', 'TRABAJO', 'TIPO DE VIVIENDA',
       'AÑOS EN LA VIVIENDA', 'TIPO DE CONTRATO', 'PERIODO DE PAGO',
       'ESTADO CIVIL', 'NIVEL EDUCATIVO', 'TIPO DE VEHICULO',
       'TIEMPO TRABAJO'],
      dtype='object')

## Manejo datos de fecha


In [6]:
df['FECHA DESEMBOLSO'] = df['FECHA DESEMBOLSO'].str.replace('/', '-')

# Convertir las fechas al formato "%Y-%m-%d"
df['FECHA DESEMBOLSO'] = pd.to_datetime(df['FECHA DESEMBOLSO'], errors='coerce', format='%m-%d-%Y')

In [7]:
df['PRÓXIMA FECHA PAGO'] = df['PRÓXIMA FECHA PAGO'].str.replace('/', '-')

# Convertir las fechas al formato "%Y-%m-%d"
df['PRÓXIMA FECHA PAGO'] = pd.to_datetime(df['PRÓXIMA FECHA PAGO'], errors='coerce', format='%m-%d-%Y')

## Procesamiento de texto variables categricas


In [8]:
def processing_text(texto):
    if texto is None:
        return None
    texto = str(texto)
    ### Limpieza
    texto = texto.lower() # Estandarizar todo a minúscula
    texto = unidecode(texto)  # Eliminar tildes
    texto = re.sub(r'[^a-zA-Z0-9\s/-]', '', texto)
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios en blanco adicionales
    texto = texto.strip() # Eliminar espacios al inicio y al final
    
    
    return texto  

In [9]:
df['TIEMPO TRABAJO'][3719]

'Julio2-2015'

In [10]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].map(processing_text)

In [11]:
df['TIEMPO TRABAJO'][3719]

'julio2-2015'

## agrupacion de ciudades


In [12]:
mapeo = {}

for ciudad in df['CIUDAD RESIDENCIA']:
    if isinstance(ciudad, str):  # Verificar si es un string
        ciudad_estandar = None
        # Buscar si la ciudad ya está en el diccionario
        for ciudad_mapeada in mapeo:
            if fuzz.ratio(ciudad.lower(), ciudad_mapeada.lower()) > 70:  # Umbral de similitud del 80%
                ciudad_estandar = ciudad_mapeada
                break
        if ciudad_estandar:
            mapeo[ciudad_estandar].append(ciudad)
        else:
            mapeo[ciudad] = [ciudad]

# Crear una nueva columna en el DataFrame con la ciudad estandarizada
def obtener_ciudad_estandar(ciudad):
    for ciudad_estandar, variantes in mapeo.items():
        if ciudad in variantes:
            return ciudad_estandar
    return ciudad

df['ciudad_estandarizada'] = df['CIUDAD RESIDENCIA'].apply(obtener_ciudad_estandar)

## Procesamiento tiempo en trabajo


In [13]:
# Separar datos con letras y números
letras_numeros = df['TIEMPO TRABAJO'].str.contains(r'[a-zA-Z]', regex=True)
#letras_numeros = letras_numeros.fillna(False)  # Rellenar NaN con False
df['letras_numeros'] = df['TIEMPO TRABAJO'][letras_numeros]
df['letras_numeros'] = df['letras_numeros'].replace('nan', np.nan)
df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'][~letras_numeros]

## poner que si solo tiene numero agregarle la palabra año y es menor a 20


In [17]:
df['letras_numeros'][38]

nan

In [401]:
def es_fecha(texto):
    if pd.isna(texto):
        return None
    texto = re.sub(r'(\D)(\d)', r'\1 \2', texto)
    texto = re.sub(r'[^a-zA-Z0-9\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    
    
    meses = r'(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)'
    patron = rf'({meses} \d{{1,2}} de \d{{4}}|{meses} \d{{1,2}} \d{{4}}|{meses} – \d{{1,2}} – \d{{4}}|{meses}\d{{1,2}}-\d{{4}})'
    #patron = rf'({meses} \d{{1,2}} de \d{{4}}|{meses} \d{{1,2}} \d{{4}}|{meses} – \d{{1,2}} – \d{{4}}|{meses}\d{{1,2}}-\d{{4}})'
    match = re.search(patron, texto)
    if match:
        return match.group()

df['fechas'] = df['letras_numeros'].apply(es_fecha)

df['fechas'].dropna().head(30)

262       marzo 1 de 2019
315          mayo 15 2018
2895       agosto 20 2000
3198    noviembre 19 2018
3719         julio 2 2015
3985       febrero 6 2020
Name: fechas, dtype: object

In [377]:
def obtener_valor_numerico(texto):
    if texto is not np.nan:
        texto = str(texto)
        texto = texto.lower()
        meses = 0
        anios = 0
        if 'mes' in texto:
            meses = re.findall(r'(\d+)\s*(?:mes?|meses?)', texto)
            if len(meses)==0:
                meses = 1
            else:    
                meses = int(meses[0])
                
        if any(word in texto.lower() for word in ['años', 'anos', 'año', 'años', 'ano']):
            anios = re.findall(r'(\d+)\s*(?:años?|anos?|año)', texto)
            if len(anios)==0:
                anios = 12
            else:    
                anios = 12*int(anios[0])
        if 'medio' in texto:
            meses = meses + 6
        return meses + anios
    return np.nan

In [378]:
df['letras_numeros'] = df['letras_numeros'].apply(obtener_valor_numerico)
df['letras_numeros'].head()

0    24.0
1    12.0
2    12.0
3     NaN
4     0.0
Name: letras_numeros, dtype: float64

In [411]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

df['fechas'] = df['fechas'].str.replace(' de ', ' ').str.capitalize()
df['fechas2'] = pd.to_datetime(df['fechas'], format='%B %d %Y') 
df['fechas2'] = df['fechas2'].dt.strftime('%m-%d-%Y')

# Mostrar el DataFrame resultante
df['fechas2'].dropna().head(30)

262     03-01-2019
315     05-15-2018
2895    08-20-2000
3198    11-19-2018
3719    07-02-2015
3985    02-06-2020
Name: fechas2, dtype: object

In [ ]:
# sumar dias desde fecha trabajo hasta fecha desembolso

In [ ]:
# unir todo enuna sola columna